Imports

In [ ]:
!pip install -q transformers
!pip install -U tensorflow==2.11 --quiet
import pandas as pd
import json
import numpy as np
from google.colab import drive


import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds


import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re
from sklearn.model_selection import train_test_split
import string
import ast
from keras import backend as K


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 85.0 MB/s eta 0:00:00


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFRobertaModel, TFAutoModelForSequenceClassification, AutoTokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = TFRobertaModel.from_pretrained('roberta-base',output_hidden_states=True, output_attentions=True)


In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model2 = TFAutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", output_hidden_states=True)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
drive.mount('/content/gdrive')
sentfin = pd.read_csv('/content/gdrive/MyDrive/Raw Data/SEntFiN-v1.1.csv')

In [ ]:
from keras.layers import Input, Flatten

In [ ]:


from sklearn.model_selection import train_test_split
sentfin = pd.read_csv('/content/gdrive/MyDrive/Raw Data/SEntFiN-v1.1.csv')


X = np.arange(0,10753)
y = np.arange(0,10753)
x_train, x_test, y_train, y_test = train_test_split(sentfin['Title'], sentfin['Decisions'], test_size=0.3,  random_state=8)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.6660, random_state=8)

In [ ]:
len(x_val), len(y_val), len(x_test), len(y_test), len(y_train), len(x_train)

(2149, 2149, 1077, 1077, 7527, 7527)

In [ ]:
x_train[10680] = 'Global oil worries, rupee fall weigh on stocks'



y_train[220] = '{"Nifty50": "neutral"}'
y_train[792] =  '{"Alibaba": "neutral"}'
y_train[6056] =  '{"IIHFL": "neutral"}'
y_train[10701] = '{"Australia": "negative", "NZ dlrs": "negative"}'
y_train[9451] = '{"Gold": "negative"}'
y_train[9181] = '{"Brent": "negative"}'


y_train[10166] = '{"Bharti": "positive", "Tata Comm": "positive", "mobile content players": "positive"}'



## Tokenizing the Input

In [ ]:
### Function Helps catch all special cases in the tokenization function

def return_helper(ex_title, ex):
    punctuations = [':', ',', "'s", ';', '?', "'", 's', "s'"]

    for punctuation in punctuations:
        pattern = re.escape(ex + punctuation)
        match = re.search(pattern, ex_title)
        if match:
            return match.start()

    return None



In [ ]:
def clean_data2(x_data,y_data, model = model, tokenizer = tokenizer):
  '''input: news headline data set with sentiments
  output: roberta tokenized inputs, and corresponding label
  '''

  title =[str(i) for i in x_data.values]
  headlines = []
  delete = []

  for row in title:
    headline = " ".join("".join([" " if ch in string.punctuation else ch for ch in row]).split())
    headlines.append(headline)
  try:
    decisions = ([json.loads(i) for i in y_data])
  except:
    decisions = (([ast.literal_eval(i) for i in y_data]))
  entities = []
  sents = []
  for ex in decisions:
    r_entities = []
    r_sents = []
    for key in ex.keys():
      r_entities.append(" ".join("".join([" " if ch in string.punctuation else ch for ch in key]).split()))
    for value in ex.values():
      r_sents.append(value)
    entities.append(r_entities)
    sents.append(r_sents)

  title_tokenizer = tokenizer(headlines, padding='max_length', max_length=30, truncation=True, return_tensors="tf")
  text_tokenized = tokenizer.batch_encode_plus(headlines, padding='max_length', max_length=30, truncation=True, return_tensors="tf")

  problems = 0
  i = 0
  fin_labels = []
  while i < len(y_data):


    ex_title = headlines[i].split()


    indexes = []

    input = np.array(text_tokenized.word_ids(i))

    for ex_entity in entities[i]:
      word_ent = []
      ex = ex_entity.split()[0]
      try:
        return_index = ex_title.index(ex)

        matches = [j for j,val in enumerate(input) if val==return_index]

        for matching in matches:
          word_ent.append(matching)

      except:
        helper = return_helper(ex_title, ex)
        if helper is None:
          problems = problems + 1

          delete.append(i)
          continue
        return_index = helper
        matches = [j for j,val in enumerate(input) if val==return_index]
        for matching in matches:
          word_ent.append(matching)
      if len(ex_entity.split()) > 1:
        for ex in ex_entity.split()[1:]:

          return_index = return_index + 1
          matches = [j for j,val in enumerate(input) if val==return_index]
          for matching in matches:
              word_ent.append(matching)



      indexes.append(word_ent)

    return_label = np.zeros(30)
    z = 0
    while z < len(indexes):
      x = 0
      while x < len(indexes[z]):
        if x == 0:

          if sents[i][z] == 'positive':
            return_label[indexes[z][x]] = 4
          elif sents[i][z] == 'neutral':
            return_label[indexes[z][x]] = 3
          else:
            return_label[indexes[z][x]] = 2
        else:
          return_label[indexes[z][x]] = 1
        x = x + 1
      z = z + 1



    fin_labels.append(return_label)
    i = i+1


  labels = tf.convert_to_tensor(fin_labels, dtype=tf.float32)


  dataset = tf.data.Dataset.from_tensor_slices((
      dict(title_tokenizer),
      labels
  ))

  return dataset, dict(title_tokenizer), labels, delete,  title , headlines, decisions

In [ ]:
train_dataset, train_tokens, trained_labels,train_delete, train_sentence, train_headline, train_decision = clean_data2(x_train,y_train,model, tokenizer)


test_dataset, test_tokens, test_labels, test_delete, test_sentence, test_headline, test_decision= clean_data2(x_test,y_test,model, tokenizer)
val_dataset, val_tokens, valid_labels,val_delete, val_sentence, val_headline, val_decision  = clean_data2(x_val, y_val,model, tokenizer)

In [ ]:
from keras.utils.np_utils import to_categorical

train_labels = to_categorical(trained_labels, num_classes=5)
train_labels = tf.convert_to_tensor(train_labels )

val_labels = to_categorical(valid_labels, num_classes=5)
val_labels = tf.convert_to_tensor(val_labels )

test_labels = to_categorical(test_labels, num_classes=5)
test_labels = tf.convert_to_tensor(test_labels )

## Roberta Model (used for standard and w/ pretrain function)



In [ ]:
max_length = 30
def create_roberta_multiclass_model(model = model,
                                 num_classes = 5,
                                 hidden_size = 30,
                                 dropout=0.2,
                                 learning_rate=0.00005):

    keras.backend.clear_session()
    bert_model = model


    bert_model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask')

    bert_inputs = {'input_ids': input_ids,
                  'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    output = bert_out['last_hidden_state']


    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_2')(output)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    hidden2 = tf.keras.layers.Dense(hidden_size/3, activation='relu', name='hidden_layer_3')(hidden)

    hidden2 = tf.keras.layers.Dropout(dropout)(hidden2)

    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden2)

    classification_model = tf.keras.Model(inputs=[input_ids,attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                metrics=['accuracy', tf.keras.metrics.Precision(thresholds=0, class_id = 2), ])

    return classification_model

In [ ]:


roberta_model = create_roberta_multiclass_model( num_classes=5, hidden_size=120, dropout = 0.15)




In [ ]:
roberta_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['attention_mask[0][0]',         
 el)                            thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                            

In [ ]:

roberta_model_history = roberta_model.fit(train_tokens, train_labels,

                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=16,
                                                  epochs=6
                                                  )

Epoch 1/6


471/471 [==============================] - 143s 193ms/step - loss: 0.0674 - accuracy: 0.9417 - precision: 0.0119 - val_loss: 0.0341 - val_accuracy: 0.9656 - val_precision: 0.0114
Epoch 2/6
471/471 [==============================] - 78s 166ms/step - loss: 0.0343 - accuracy: 0.9701 - precision: 0.0119 - val_loss: 0.0255 - val_accuracy: 0.9791 - val_precision: 0.0114
Epoch 3/6
471/471 [==============================] - 76s 161ms/step - loss: 0.0248 - accuracy: 0.9810 - precision: 0.0119 - val_loss: 0.0233 - val_accuracy: 0.9795 - val_precision: 0.0114
Epoch 4/6
471/471 [==============================] - 76s 161ms/step - loss: 0.0203 - accuracy: 0.9843 - precision: 0.0119 - val_loss: 0.0331 - val_accuracy: 0.9769 - val_precision: 0.0114
Epoch 5/6
471/471 [==============================] - 76s 161ms/step - loss: 0.0217 - accuracy: 0.9836 - precision: 0.0119 - val_loss: 0.0296 - val_accuracy: 0.9799 - val_precision: 0.0114
Epoch 6/6
471/471 [==============================] - 75s 160ms/step -



##FINBERT
```



In [ ]:
max_length = 30
def create_finbert_multiclass_model(model = model,
                                 num_classes = 5,
                                 hidden_size = 30,
                                 dropout=0.2,
                                 learning_rate=0.00005):

    keras.backend.clear_session()
    bert_model = model


    bert_model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids')

    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')

    bert_inputs = {'input_ids': input_ids,
                  'attention_mask': attention_mask}
                  # 'token_type_ids_layer': token_type_ids}




    bert_out = bert_model(bert_inputs)

    output = bert_out['hidden_states'][12]


    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_2')(output)

    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    hidden2 = tf.keras.layers.Dense(hidden_size/3, activation='relu', name='hidden_layer_3')(hidden)

    hidden2 = tf.keras.layers.Dropout(dropout)(hidden2)

    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden2)

    classification_model = tf.keras.Model(inputs=[input_ids,attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                metrics=['accuracy', tf.keras.metrics.Precision(thresholds=0, class_id = 2), ])

    return classification_model

In [ ]:
finbert_model = create_finbert_multiclass_model( num_classes=5, hidden_size=120, dropout = 0.2,model = model2)
finbert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_for_sequence_classific  TFSequenceClassifie  109484547  ['attention_mask[0][0]',         
 ation (TFBertForSequenceClassi  rOutput(loss=None,               'input_ids[0][0]']              
 fication)                      logits=(None, 3),                                                 
                                 hidden_states=((No                                           

In [ ]:
finbert_history = finbert_model.fit(train_tokens, train_labels,

                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=16,
                                                  epochs=5
                                                  )

Epoch 1/5


/usr/local/lib/python3.9/dist-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


471/471 [==============================] - 147s 191ms/step - loss: 0.0688 - accuracy: 0.9407 - precision: 0.0119 - val_loss: 0.0321 - val_accuracy: 0.9661 - val_precision: 0.0114
Epoch 2/5
471/471 [==============================] - 76s 160ms/step - loss: 0.0318 - accuracy: 0.9719 - precision: 0.0119 - val_loss: 0.0276 - val_accuracy: 0.9769 - val_precision: 0.0114
Epoch 3/5
471/471 [==============================] - 74s 158ms/step - loss: 0.0210 - accuracy: 0.9838 - precision: 0.0119 - val_loss: 0.0298 - val_accuracy: 0.9779 - val_precision: 0.0114
Epoch 4/5
471/471 [==============================] - 73s 154ms/step - loss: 0.0155 - accuracy: 0.9886 - precision: 0.0119 - val_loss: 0.0303 - val_accuracy: 0.9777 - val_precision: 0.0114
Epoch 5/5
471/471 [==============================] - 73s 155ms/step - loss: 0.0115 - accuracy: 0.9917 - precision: 0.0119 - val_loss: 0.0352 - val_accuracy: 0.9793 - val_precision: 0.0114


In [ ]:
tf.keras.backend.clear_session()


In [ ]:
y_pred = roberta_model.predict(val_tokens)

68/68 [==============================] - 8s 75ms/step


In [ ]:
y_pred = finbert_model.predict(val_tokens)

68/68 [==============================] - 13s 89ms/step


In [ ]:
trials = []
for example in y_pred:
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())

  trials.append(trial)













#513, 477, 458

In [ ]:

trials_one = []
for example in val_labels.numpy():
  trial_one = []
  for row in example:
    trial_one.append(tf.argmax(row, 0, name=None).numpy())


  trials_one.append(trial_one)


test = 0
count = 0


In [ ]:
test = 0
incorrect = 0
correct = 0
prediction = []
trial = []
while test < len(val_labels):

  if trials_one[test] != trials[test]:
    incorrect = incorrect +1
    #print("Incorrect: " + str(incorrect))
    #print(tokenizer.decode(val_tokens['input_ids'][test]))
    #print("Incorrect Label:")
    #print(np.argmax(val_labels[test] , axis = 1))
    #print("Incorrect Prediction:")
    #print(trials[test])
    #print(val_decision[test])
    prediction.append(0)

  if trials_one[test] == trials[test]:
    correct = correct +1
    prediction.append(1)
  test = test + 1
print(incorrect)
print(correct)

728
1421


In [ ]:
1421/(1505+644)

0.6612377850162866

Parsing additional data for Pre-Train

In [ ]:
semval = pd.read_csv('/content/gdrive/MyDrive/Raw Data/semval_cleaned.csv')
semval_r = semval[["title", "Decisions"]]
fiqa = pd.read_csv('/content/gdrive/MyDrive/Raw Data/fiqa_cleaned.csv')
fiqa_r = fiqa[["title", "Decisions"]]



pre_train_data = pd.concat([fiqa_r,semval_r],ignore_index = True)


pretrain_dataset, pretrain_tokens, pretrain_labels, delete, pretrain_sentence, pretrain_headline, pretrain_decision = clean_data2(pre_train_data['title'],pre_train_data['Decisions'])


pre_train_data.drop(delete, axis=0, inplace=True)

pretrain_dataset, pretrain_tokens, pretrain_labels, delete, pretrain_sentence, pretrain_headline, pretrain_decision= clean_data2(pre_train_data['title'],pre_train_data['Decisions'])


pretrain_labels = to_categorical(pretrain_labels, num_classes=5)
pretrain_labels = tf.convert_to_tensor(pretrain_labels )


In [ ]:
len(semval_r),len(fiqa_r),len(pre_train_labels)

(966, 436)

##Pre-Train model


In [ ]:

tf.keras.backend.clear_session()
roberta_model_pt = create_roberta_multiclass_model(num_classes=5, hidden_size=120, dropout = 0.2)


In [ ]:

roberta_model_pt.fit(pretrain_tokens, pretrain_labels,
                                                  batch_size=16,
                                                  epochs= 2)


Epoch 1/2


80/80 [==============================] - 55s 159ms/step - loss: 0.1654 - accuracy: 0.8788 - precision: 0.0114
Epoch 2/2
80/80 [==============================] - 7s 81ms/step - loss: 0.0761 - accuracy: 0.9397 - precision: 0.0114


In [ ]:

roberta_model_pt_history = roberta_model_pt.fit(train_tokens, train_labels,

                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=16,
                                                  epochs= 4)



Epoch 1/4
471/471 [==============================] - 38s 80ms/step - loss: 0.0508 - accuracy: 0.9551 - precision: 0.0119 - val_loss: 0.0290 - val_accuracy: 0.9750 - val_precision: 0.0114
Epoch 2/4
471/471 [==============================] - 31s 66ms/step - loss: 0.0300 - accuracy: 0.9770 - precision: 0.0119 - val_loss: 0.0271 - val_accuracy: 0.9765 - val_precision: 0.0114
Epoch 3/4
471/471 [==============================] - 29s 62ms/step - loss: 0.0226 - accuracy: 0.9824 - precision: 0.0119 - val_loss: 0.0263 - val_accuracy: 0.9788 - val_precision: 0.0114
Epoch 4/4
471/471 [==============================] - 29s 62ms/step - loss: 0.0179 - accuracy: 0.9864 - precision: 0.0119 - val_loss: 0.0299 - val_accuracy: 0.9795 - val_precision: 0.0114


In [ ]:
y_pred_pretrain = roberta_model_pt.predict(val_tokens)


test = 0
count = 0
trials_p = []
for example in y_pred_pretrain:
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())

  trials_p.append(trial)


68/68 [==============================] - 5s 25ms/step


In [ ]:
test = 0
incorrect_p = 0
correct_p = 0
pred_p = []
while test < len(val_labels):
  if trials_one[test] == trials_p[test]:
    correct_p = correct_p +1
    pred_p.append(1)

  if trials_one[test] != trials_p[test]:
    incorrect_p = incorrect_p +1
    pred_p.append(0)
  test = test + 1
print(correct_p)

NameError: ignored

##Pre-Train model

In [ ]:

tf.keras.backend.clear_session()
roberta_model_pt_2 = create_roberta_multiclass_model( num_classes=5, hidden_size=120, dropout = 0.15)

roberta_model_pt_2.fit(pretrain_tokens, pretrain_labels,
                                                  batch_size=16,
                                                  epochs=2)
roberta_model_pt_history_2 = roberta_model_pt_2.fit(train_tokens, train_labels,
                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=16,
                                                  epochs=4)

In [ ]:
predictions = pd.read_csv('/content/gdrive/MyDrive/Raw Data/predictions/roberta_pt.csv')

In [ ]:
from sklearn.metrics import classification_report



In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 16.0 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa
from tensorflow_addons.layers import CRF

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


##Implementing CRF for Roberta

In [ ]:
max_length = 30



def create_roberta_crf_model(model = model,
                                 num_classes = 5,
                                 hidden_size = 30,
                                 dropout=0.2,
                                 learning_rate = 0.00005):

    keras.backend.clear_session()
    bert_model = model




    bert_model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask')

    bert_inputs = {'input_ids': input_ids,
                  'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)




    output_layer_1 = bert_out['hidden_states'][11]

    hidden_1 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_1')(output_layer_1)
    hidden_1 = tf.keras.layers.Dropout(dropout)(hidden_1)
    decoded_sequence, classification_1, sequence_length, chain_kernel= tfa.layers.CRF(5)(hidden_1)
    classification_1 = tf.keras.layers.Dense(5, activation='softmax',name='classification_layer_1')(classification_1)


    output_layer_2 = bert_out['hidden_states'][10]
    hidden_2 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_2')(output_layer_2)
    hidden_2 = tf.keras.layers.Dropout(dropout)(hidden_2)
    decoded_sequence, classification_2, sequence_length, chain_kernel= tfa.layers.CRF(5)(hidden_2)
    classification_2 = tf.keras.layers.Dense(5, activation='softmax',name='classification_layer_2')(classification_2)

    print(classification_2)
    output_layer_3 = bert_out['hidden_states'][9]
    hidden_3 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_3')(output_layer_3)
    hidden_3 = tf.keras.layers.Dropout(dropout)(hidden_3)
    decoded_sequence, classification_3, sequence_length, chain_kernel= tfa.layers.CRF(5)(hidden_3)
    classification_3 = tf.keras.layers.Dense(5, activation='softmax',name='classification_layer_3')(classification_3)




    output_layer_4 = bert_out['hidden_states'][8]
    hidden_4 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer_4')(output_layer_4)
    hidden_4= tf.keras.layers.Dropout(dropout)(hidden_4)
    decoded_sequence, classification_4, sequence_length, chain_kernel= tfa.layers.CRF(5)(hidden_4)
    classification_4 = tf.keras.layers.Dense(5, activation='softmax',name='classification_layer_4')(classification_4)






    concatted = tf.keras.layers.Average()([classification_1, classification_2, classification_3, classification_4])

    classification_model = tf.keras.Model(inputs=[input_ids,attention_mask], outputs=[concatted])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                metrics=['accuracy', tf.keras.metrics.Precision(thresholds=0, class_id = 2), ])

    return classification_model

In [ ]:
tf.keras.backend.clear_session()



Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


KerasTensor(type_spec=TensorSpec(shape=(None, 30, 5), dtype=tf.float32, name=None), name='classification_layer_2/Softmax:0', description="created by layer 'classification_layer_2'")


In [ ]:
tf.keras.backend.clear_session()


roberta_crf = create_roberta_crf_model(num_classes=5, hidden_size=120, dropout = 0.20)


roberta_crf_history = roberta_crf.fit(train_tokens, train_labels,
                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=32,

                                                  epochs=5
                                                  )

KerasTensor(type_spec=TensorSpec(shape=(None, 30, 5), dtype=tf.float32, name=None), name='classification_layer_2/Softmax:0', description="created by layer 'classification_layer_2'")
Epoch 1/5


236/236 [==============================] - 77s 132ms/step - loss: 0.0191 - accuracy: 0.9872 - precision: 0.0119 - val_loss: 0.0307 - val_accuracy: 0.9802 - val_precision: 0.0114
Epoch 2/5
236/236 [==============================] - 16s 70ms/step - loss: 0.0057 - accuracy: 0.9956 - precision: 0.0119 - val_loss: 0.0322 - val_accuracy: 0.9803 - val_precision: 0.0114
Epoch 3/5
236/236 [==============================] - 16s 70ms/step - loss: 0.0041 - accuracy: 0.9967 - precision: 0.0119 - val_loss: 0.0345 - val_accuracy: 0.9813 - val_precision: 0.0114
Epoch 4/5
236/236 [==============================] - 16s 67ms/step - loss: 0.0033 - accuracy: 0.9974 - precision: 0.0119 - val_loss: 0.0395 - val_accuracy: 0.9814 - val_precision: 0.0114
Epoch 5/5
236/236 [==============================] - 16s 67ms/step - loss: 0.0032 - accuracy: 0.9974 - precision: 0.0119 - val_loss: 0.0375 - val_accuracy: 0.9812 - val_precision: 0.0114


In [ ]:

roberta_crf_2 = create_roberta_crf_model(num_classes=5, hidden_size=120, dropout = 0.20)

roberta_crf_2.fit(pretrain_tokens, pretrain_labels,
                                                  batch_size=16,
                                                  epochs=2)


roberta_crf_2_history = roberta_crf_2.fit(train_tokens, train_labels,
                                                  validation_data=(val_tokens, val_labels),
                                                  batch_size=32,

                                                  epochs=4
                                                  )

KerasTensor(type_spec=TensorSpec(shape=(None, 30, 5), dtype=tf.float32, name=None), name='classification_layer_2/Softmax:0', description="created by layer 'classification_layer_2'")
Epoch 1/2


80/80 [==============================] - 71s 193ms/step - loss: 0.0990 - accuracy: 0.9151 - precision: 0.0114
Epoch 2/2
80/80 [==============================] - 11s 139ms/step - loss: 0.0472 - accuracy: 0.9543 - precision: 0.0114
Epoch 1/4
236/236 [==============================] - 75s 315ms/step - loss: 0.0147 - accuracy: 0.9883 - precision: 0.0119 - val_loss: 0.0269 - val_accuracy: 0.9795 - val_precision: 0.0114
Epoch 2/4
236/236 [==============================] - 57s 239ms/step - loss: 0.0097 - accuracy: 0.9917 - precision: 0.0119 - val_loss: 0.0279 - val_accuracy: 0.9808 - val_precision: 0.0114
Epoch 3/4
236/236 [==============================] - 56s 238ms/step - loss: 0.0080 - accuracy: 0.9932 - precision: 0.0119 - val_loss: 0.0286 - val_accuracy: 0.9812 - val_precision: 0.0114
Epoch 4/4
236/236 [==============================] - 56s 236ms/step - loss: 0.0065 - accuracy: 0.9945 - precision: 0.0119 - val_loss: 0.0305 - val_accuracy: 0.9805 - val_precision: 0.0114


In [ ]:
y_crf_new= roberta_crf_2.predict(val_tokens)

68/68 [==============================] - 19s 86ms/step


In [ ]:
trials = []
for example in y_crf_new:
  trial = []
  for row in example:
    trial.append(tf.argmax(row, 0, name=None).numpy())

  trials.append(trial)

In [ ]:
test = 0
incorrect = 0
correct = 0
prediction = []
trial = []
while test < len(val_labels):

  if trials_one[test] != trials[test]:
    incorrect = incorrect +1
    #print()
    #print("Incorrect: " + str(incorrect))
    #print(tokenizer.decode(val_tokens['input_ids'][test]))
    #print("Incorrect Label:")
    #print(np.argmax(val_labels[test] , axis = 1))
    #print("Incorrect Prediction:")
    #print(trials[test])
    #print(val_decision[test])
    prediction.append(0)

  if trials_one[test] == trials[test]:
    correct = correct +1
    #print()
    #print("Correct: " + str(correct))
    #print(tokenizer.decode(val_tokens['input_ids'][test]))
    #print("Correct Label:")
    #print(np.argmax(val_labels[test] , axis = 1))
    #print("Correct Prediction:")
    #print(trials[test])
    #print(val_decision[test])
    prediction.append(1)
  test = test + 1
print(incorrect)
print(correct)

664
1485


In [ ]:
y_true= tf.reshape(tf.argmax(val_labels, 2), 2149*30)


orig_predictions = tf.reshape(tf.argmax(y_pred, 2), 2149*30)
target_names = ['class 0', 'words', 'negative','nuetral','positive']

print("Classification Report: Example for Original Predictions")
print(classification_report(y_true, orig_predictions, target_names=target_names))

Classification Report: Example for Original Predictions
              precision    recall  f1-score   support

     class 0       0.99      0.99      0.99     56916
       words       0.92      0.96      0.94      4726
    negative       0.80      0.84      0.82       732
     nuetral       0.80      0.75      0.77      1098
    positive       0.85      0.85      0.85       998

    accuracy                           0.98     64470
   macro avg       0.87      0.88      0.87     64470
weighted avg       0.98      0.98      0.98     64470



In [ ]:
y_true= tf.reshape(tf.argmax(val_labels, 2), 2149*30)


pre_train_pred = tf.reshape(tf.argmax(y_pred_pretrain, 2), 2149*30)



target_names = ['class 0', 'words', 'negative','nuetral','positive']

print("Classification Report: Example for Pred Pre Train")
print(classification_report(y_true, pre_train_pred, target_names=target_names))



Classification Report: Example for Pred Pre Train
              precision    recall  f1-score   support

     class 0       0.99      0.99      0.99     56916
       words       0.92      0.95      0.94      4726
    negative       0.81      0.84      0.82       732
     nuetral       0.76      0.76      0.76      1098
    positive       0.81      0.85      0.83       998

    accuracy                           0.98     64470
   macro avg       0.86      0.88      0.87     64470
weighted avg       0.98      0.98      0.98     64470



In [ ]:
crf_pred = tf.reshape(tf.argmax(y_crf_new, 2), 2149*30)



target_names = ['class 0', 'words', 'negative','nuetral','positive']

print("Classification Report: Example for CRF")
print(classification_report(y_true, crf_pred, target_names=target_names))

Classification Report: Example for CRF
              precision    recall  f1-score   support

     class 0       0.99      0.99      0.99     56916
       words       0.93      0.95      0.94      4726
    negative       0.85      0.83      0.84       732
     nuetral       0.79      0.80      0.80      1098
    positive       0.85      0.86      0.86       998

    accuracy                           0.98     64470
   macro avg       0.88      0.89      0.89     64470
weighted avg       0.98      0.98      0.98     64470



In [ ]:
crf_pred = tf.reshape(tf.argmax(y_crf_new, 2), 1613*30)

y_true= tf.reshape(tf.argmax(val_labels, 2), 1613*30)

target_names = ['class 0', 'words', 'negative','nuetral','positive']

print("Classification Report: Example for CRF + Pre-Train")
print(classification_report(y_true, crf_pred, target_names=target_names))

In [ ]:
# = tf.argmax(roberta_crf.predict(val_tokens),2).numpy()
#crf_test = tf.argmax(roberta_crf.predict(test_tokens),2).numpy()

#pt_val = tf.argmax(roberta_model_pt.predict(val_tokens),2).numpy()
#pt_test = tf.argmax(roberta_model_pt.predict(test_tokens),2).numpy()

#val = tf.argmax(roberta_model.predict(val_tokens),2).numpy()
#test = tf.argmax(roberta_model.predict(test_tokens),2).numpy()

finbert_v = tf.argmax(finbert_model.predict(val_tokens),2).numpy()
finbert_t = tf.argmax(finbert_model.predict(test_tokens),2).numpy()



34/34 [==============================] - 4s 107ms/step


In [ ]:
to_csv = []
for prediction in finbert_t:
  to_csv.append([prediction])

to_csv[0]

[array([0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])]

In [ ]:


pd.DataFrame(to_csv, columns=['predictions']).to_csv('/content/gdrive/MyDrive/Raw Data/predictions/finbert_test.csv')


In [ ]:
to_csv = []
for prediction in pt_test:
  to_csv.append([prediction])

to_csv[6]
pd.DataFrame(to_csv, columns=['predictions']).to_csv('/content/gdrive/MyDrive/Raw Data/predictions/base_pt_testF.csv')

In [ ]:
pt_test

array([[0, 4, 1, ..., 0, 0, 0],
       [0, 4, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 3, 1, ..., 0, 0, 0]])